# Werkcollege-opdrachten Week 1.3

## Dependencies importeren

Kopieer in het codeblok hieronder van het vorige practicum de import-code voor de dependencies die het vaakst worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
Zet eventuele warnings uit.

In [3]:
import warnings
warnings.simplefilter('ignore')
import pandas
import sqlite3

Zet het bestand go_sales_train.sqlite in een makkelijk te vinden map

## Databasetabellen inlezen

Kopieer in het codeblok hieronder van het vorige practicum de code om een connectie met het bestand go_sales_train.sqlite te maken.

In [4]:
conn = sqlite3.connect("../data/go_sales_train.sqlite")
print(conn)

Lees van de ingelezen go_sales_train-database te volgende tabellen in met behulp van "SELECT * FROM *tabel*".
- product
- product_type
- product_line
- sales_staff
- sales_branch
- retailer_site
- country
- order_header
- order_details
- returned_item
- return_reason

In [5]:
tables_query = "SELECT name FROM sqlite_master WHERE type= 'table';"
tables = pandas.read_sql(tables_query, conn)

df = {}

for table_name in tables['name']:
    query = f"SELECT * FROM {table_name}"
    df[table_name] = pandas.read_sql(query, conn)

def get_table_data(table_names):
    
    if isinstance(table_names, str):
        table_names = [table_names]

    tables = [df.get(name, pandas.DataFrame()) for name in table_names]

    if not tables:
        print(f"None of the specified tables were found.")
        return pandas.DataFrame()

    merged_data = merge_table_data(tables)
    return merged_data

def merge_table_data(tables):
    merged_data = tables[0]

    for current_table in tables[1:]:
        common_columns = merged_data.columns.intersection(current_table.columns)

        if not common_columns.empty:
            merge_column = common_columns[0]
            merged_data = pandas.merge(merged_data, current_table, on=merge_column, how='inner')
        else:
            print(f"No common column found between the tables")

    return merged_data

Krijg je een "no such table" error? Dan heb je misschien met .connect() per ongeluk een leeg  databasebestand (.sqlite) aangemaakt. <u>Let op:</u> lees eventueel de informatie uit het Notebook van werkcollege 1.1b nog eens goed door.

Als je tijdens onderstaande opdrachten uit het oog verliest welke tabellen er allemaal zijn, kan je deze Pythoncode uitvoeren:

In [6]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
#Vul dit codeblok verder in
pandas.read_sql(sql_query, conn)
#Op de puntjes hoort de connectie naar go_sales_train óf go_staff_train óf go_crm_train te staan.

,name
0,country
1,order_details
2,order_header
3,order_method
4,product
5,product_line
6,product_type
7,retailer_site
8,return_reason
9,returned_item


erachter 

Let op! Voor alle onderstaande opdrachten mag je <u>alleen Python</u> gebruiken, <u>geen SQL!</u>

## Selecties op één tabel zonder functies

Geef een overzicht met daarin de producten en hun productiekosten waarvan de productiekosten lager dan 100 dollar en hoger dan 50 dollar ligt. (2 kolommen, 23 rijen)

In [7]:
table_data = get_table_data('product')

filter = (table_data['PRODUCTION_COST'] > 50) & (table_data['PRODUCTION_COST'] < 100)
table_data.loc[filter, ['PRODUCT_NAME', 'PRODUCTION_COST']]

,PRODUCT_NAME,PRODUCTION_COST
5,TrailChef Deluxe Cook Set,85.11
7,TrailChef Double Flame,75.00
16,Hibernator Lite,60.00
17,Hibernator,86.00
19,Hibernator Self - Inflating Mat,78.55
22,Hibernator Camp Cot,65.33
23,Canyon Mule Climber Backpack,62.50
45,Granite Climbing Helmet,52.86
47,Husky Harness Extreme,55.00
57,Granite Hammer,57.13


Geef een overzicht met daarin de producten en hun marge waarvan de marge lager dan 20 % of hoger dan 60 % ligt. (2 kolommen, 7 rijen) 

In [8]:
filter = (table_data['MARGIN'] < 0.20) | (table_data['MARGIN'] > 0.60)
table_data.loc[filter, ['PRODUCT_NAME', 'MARGIN']]

,PRODUCT_NAME,MARGIN
20,Hibernator Pad,0.17
23,Canyon Mule Climber Backpack,0.17
85,BugShield Natural,0.67
86,BugShield Spray,0.67
87,BugShield Lotion Lite,0.70
88,BugShield Lotion,0.63
89,BugShield Extreme,0.63


Geef een overzicht met daarin de landen waar met francs wordt betaald. Sorteer de uitkomst op land.  (1 kolom, 3 rijen)

In [9]:
table_data = get_table_data('country')

filter = table_data['CURRENCY_NAME'] == 'francs'
table_data.loc[filter, ['COUNTRY']]

,COUNTRY
0,France
7,Switzerland
15,Belgium


Geef een overzicht met daarin de verschillende introductiedatums waarop producten met meer dan 50% marge worden geïntroduceerd (1 kolom, 7 rijen) 

In [10]:
table_data = get_table_data('product')

filter = table_data['MARGIN'] > 0.50
table_data.loc[filter, ['INTRODUCTION_DATE']].drop_duplicates('INTRODUCTION_DATE')

,INTRODUCTION_DATE
66,2000-10-26
85,1995-02-15
101,2003-12-10
103,2003-12-18
104,2003-12-27
106,2004-01-13
110,2003-12-15


Geef een overzicht met daarin het eerste adres en de stad van verkoopafdelingen waarvan zowel het tweede adres als de regio bekend is (2 kolommen, 7 rijen)

In [11]:
table_data = get_table_data('sales_branch')

filter = table_data['REGION'].notna() & table_data['ADDRESS2'].notna()
table_data.loc[filter, ['ADDRESS1', 'CITY']]

,ADDRESS1,CITY
2,Singelgravenplein 4,Amsterdam
13,Prol. Paseo de la Reforma No. 51,Distrito Federal
14,202-2-3 Hyakunincho,Tokyo
15,543-225 Asahi,Osaka City
16,2315 Queen's Ave,Melbourne
18,"Avenida Paulista, 333",São Paulo
24,3 Albany Court,Birmingham


Geef een overzicht met daarin de landen waar met dollars (dollars of new dollar) wordt betaald. Sorteer de uitkomst op land. (1 kolom, 4 rijen) 

In [12]:
table_data = get_table_data('country')

filter = table_data['CURRENCY_NAME'].str.contains('dollar')
table_data.loc[filter, ['COUNTRY']].sort_values(by='COUNTRY', ascending=True)

,COUNTRY
14,Australia
3,Canada
11,Taiwan
2,United States


Geef een overzicht met daarin beide adressen en de stad van vestigingen van klanten waarvan de postcode begint met een ‘D’ (van duitsland). Filter op vestigingen die een tweede adres hebben. (3 kolommen, 2 rijen) 

In [13]:
table_data = get_table_data('retailer_site')

filter = (table_data['POSTAL_ZONE'].str[0] == 'D') & table_data['ADDRESS2'].notna()
table_data.loc[filter, ['ADDRESS1', 'ADDRESS2', 'CITY']]

,ADDRESS1,ADDRESS2,CITY
31,Röntgenstraße 90,3. Tür rechts,Frankfurt
59,Grubesallee 141,4. Stock,Hamburg


## Selecties op één tabel met functies

Geef het totaal aantal producten dat is teruggebracht (1 waarde) 

In [14]:
table_data = get_table_data('returned_item')

print(table_data['RETURN_QUANTITY'].sum())

11908


Geef het aantal regio’s waarin verkoopafdelingen gevestigd zijn. (1 waarde)

In [15]:
table_data = get_table_data('sales_branch')

print(table_data['REGION'].count())

15


Maak 3 variabelen:
- Een met de laagste
- Een met de hoogste
- Een met de gemiddelde (afgerond op 2 decimalen)

marge van producten (3 kolommen, 1 rij) 

In [16]:
table_data = get_table_data('product')

lowest = table_data['MARGIN'].min()
highest = table_data['MARGIN'].max()
average = round(table_data['MARGIN'].mean(), 2)

frame = pandas.DataFrame({
    'lowest': [lowest],
    'highest': [highest],
    'average': [average]
})

frame

,lowest,highest,average
0,0.17,0.7,0.4


Geef het aantal vestigingen van klanten waarvan het 2e adres niet bekend is (1 waarde)

In [17]:
table_data = get_table_data('retailer_site')

filter = len(table_data['ADDRESS2'].isna())
filter

391

Geef de gemiddelde kostprijs van de verkochte producten waarop korting (unit_sale_price < unit_price) is verleend (1 waarde) 

In [18]:
table_data = get_table_data('order_details')

filter = table_data['UNIT_SALE_PRICE'].fillna(0) < table_data['UNIT_PRICE'].fillna(0)
avgUnitCost = table_data.loc[filter, 'UNIT_COST'].mean()
print(avgUnitCost)

127.34087927598803


Geef een overzicht met daarin het aantal medewerkers per medewerkersfunctie (2 kolommen, 7 rijen) 

In [19]:
table_data = get_table_data('sales_staff')

filter = table_data['POSITION_EN'].value_counts()
pandas.DataFrame(filter)

,count
POSITION_EN,
Level 2 Sales Representative,39
Level 3 Sales Representative,24
Branch Manager,19
Level 1 Sales Representative,13
District Manager,4
Regional Manager,2
General Manager,1


Geef een overzicht met daarin per telefoonnummer het aantal medewerkers dat op dat telefoonnummer bereikbaar is. Toon alleen de telefoonnummer waarop meer dan 4 medewerkers bereikbaar zijn. (2 kolommen, 10 rijen) 

In [20]:
table_data = get_table_data('sales_staff')

filter = table_data['WORK_PHONE'].value_counts()
pandas.DataFrame(filter[filter > 4])

,count
WORK_PHONE,
1 (305) 557-4810,6
1 (617) 268-6754,6
+31 (0)20 692 93 94,5
1 (403) 232-5986,5
+33 1 68 94 52 20,5
1 (206) 292-0012,5
1 (416) 493-5595,5
1 (310) 281-5722,5
+(61) 03 2982 4242,5


## Selecties op meerdere tabellen zonder functies

Geef een overzicht met daarin het eerste adres en de stad van vestigingen van klanten uit ‘Netherlands’ (2 kolommen, 20 rijen) 

In [21]:
table_data = get_table_data(['retailer_site', 'country'])

filter = table_data[table_data['COUNTRY'] == 'Netherlands']
filter[['ADDRESS1', 'CITY']]

,ADDRESS1,CITY
240,Beets 36,Beets
241,Perestraat 1,Deventer
242,Laan van Meerdervoort 966,Den Haag
243,Kalfjeslaan 47,Amstelveen
244,Startbaan 11,Amstelveen
245,Noorderspoorsingel 28,Groningen
246,Hogehilweg 13,Amsterdam
247,Spoorstraat 43,Varsseveld
248,Westhavenweg 8,Amsterdam
249,Westzeedijk 4,Muiden


Geef een overzicht met daarin de productnamen die tot het producttype ‘Eyewear’ behoren. (1 kolom, 5 rijen) 

In [22]:
table_data = get_table_data(['product_type', 'product'])

filter = table_data[table_data['PRODUCT_TYPE_EN'] == 'Eyewear']
pandas.DataFrame(filter['PRODUCT_NAME'])


,PRODUCT_NAME
67,Polar Sun
68,Polar Ice
69,Polar Sports
70,Polar Wave
71,Polar Extreme


Geef een overzicht met daarin alle unieke eerste adressen van klantvestigingen en de voornaam en achternaam van de verkopers die ‘Branch Manager’ zijn en aan deze vestigingen hebben verkocht (3 kolommen, 1 rij) 

In [23]:
# I don't snap

Geef een overzicht met daarin van de verkopers hun functie en indien zij iets hebben verkocht de datum waarop de verkoop heeft plaatsgevonden. Laat alleen de verschillende namen van de posities zien van de verkopers die het woord ‘Manager’ in hun positienaam hebben staan. (2 kolommen, 7 rijen) 

In [24]:
sales_staff_data = get_table_data('sales_staff')
order_header_data = get_table_data('order_header')

# merged_data = sales_staff_data.merge(order_header_data, on='SALES_STAFF_CODE', how='left')

# managers = merged_data[merged_data['POSITION_EN'].str.contains('Manager', na=False)].drop_duplicates('POSITION_EN')
# orders = merged_data[merged_data['POSITION_EN'].str.contains('Manager', na=False) & merged_data['ORDER_DATE'].notna()]

# result = pandas.concat([managers, orders], axis=0)
# result[['POSITION_EN', 'ORDER_DATE']]

tussentabel = pandas.merge(sales_staff_data, order_header_data, on = 'SALES_STAFF_CODE', how = 'left')
tussentabel.loc[tussentabel['POSITION_EN'].str.contains('Manager'), ['POSITION_EN', 'ORDER_DATE']].drop_duplicates(['POSITION_EN', 'ORDER_DATE'])

,POSITION_EN,ORDER_DATE
0,Branch Manager,NaN
320,Regional Manager,NaN
648,District Manager,NaN
3748,Branch Manager,2024-09-09
3749,Branch Manager,2024-03-13
3750,Branch Manager,2024-01-09
4186,General Manager,NaN


Geef een overzicht met daarin de verschillende namen van producten en bijbehorende namen van producttypen van de producten waarvoor ooit meer dan 750 stuks tegelijk verkocht zijn. (2 kolommen, 9 rijen) 

In [25]:
table_data = get_table_data(['product','order_details', 'product_type'])

filter = table_data[table_data['QUANTITY'] > 750]
pandas.DataFrame(filter[['PRODUCT_NAME', 'PRODUCT_TYPE_EN']].drop_duplicates())

,PRODUCT_NAME,PRODUCT_TYPE_EN
5427,Star Peg,Tents
11452,Firefly 2,Lanterns
28608,BugShield Spray,Insect Repellents
28892,BugShield Lotion Lite,Insect Repellents
29430,BugShield Lotion,Insect Repellents
29442,BugShield Extreme,Insect Repellents
30741,Sun Shelter 15,Sunscreen
31164,Sun Shelter 30,Sunscreen
31775,Sun Shield,Sunscreen


Geef een overzicht met daarin de productnamen waarvan ooit meer dan 40% korting is verleend. De formule voor korting is: (unit_price - unit_sale_price) / unit_price (1 kolom, 8 rijen) 

In [26]:
table_data = get_table_data(['product', 'order_details'])

korting = ((table_data['UNIT_PRICE'] - table_data['UNIT_SALE_PRICE']) / table_data['UNIT_PRICE']) * 100

filter = table_data[korting > 40]
pandas.DataFrame(filter['PRODUCT_NAME'].drop_duplicates())

,PRODUCT_NAME
27830,BugShield Natural
28283,BugShield Spray
28694,BugShield Lotion Lite
29127,BugShield Lotion
29464,BugShield Extreme
30562,Sun Shelter 15
30929,Sun Shelter 30
34441,Hailstorm Titanium Woods Set


Geef een overzicht met daarin de retourreden van producten waarvan ooit meer dan 90% van de aangeschafte hoeveelheid is teruggebracht (return_quantity/quantity). (1 kolom, 3 rijen) 

In [27]:
table_data = get_table_data(['returned_item', 'order_details' , 'return_reason'])

formula = (table_data['RETURN_QUANTITY'] / table_data['QUANTITY']) * 100

filter = table_data[formula > 90]
pandas.DataFrame(filter['RETURN_DESCRIPTION_EN']).drop_duplicates()

,RETURN_DESCRIPTION_EN
0,Unsatisfactory product
2,Wrong product shipped
3,Wrong product ordered


## Selecties op meerdere tabellen met functies

Geef een overzicht met daarin per producttype het aantal producten die tot dat producttype behoren. (2 kolommen, 21 rijen) 

In [28]:
table_data = get_table_data(['product', 'product_type'])

filter = table_data.groupby('PRODUCT_TYPE_EN')['PRODUCT_NUMBER'].count()
pandas.DataFrame(filter)

,PRODUCT_NUMBER
PRODUCT_TYPE_EN,
Binoculars,4
Climbing Accessories,7
Cooking Gear,10
Eyewear,5
First Aid,5
Golf Accessories,4
Insect Repellents,5
Irons,4
Knives,5


Geef een overzicht met daarin per land het aantal vestigingen van klanten die zich in dat land bevinden. (2 kolommen, 21 rijen) 

In [29]:
table_data = get_table_data(['retailer_site', 'country'])

filter = table_data.groupby('COUNTRY')['RETAILER_SITE_CODE'].count()
pandas.DataFrame(filter)

,RETAILER_SITE_CODE
COUNTRY,
Australia,10
Austria,10
Belgium,10
Brazil,1
Canada,31
China,10
Denmark,6
Finland,5
France,29


Geef een overzicht met daarin van de producten behorend tot het producttype ‘Cooking Gear’ per productnaam de totaal verkochte hoeveelheid en de gemiddelde verkoopprijs. Sorteer de uitkomst op totaal verkochte hoeveelheid. (4 kolommen, 10 rijen) 

In [30]:

table_data = get_table_data(['product', 'order_details', 'product_type'])
filtered_data = table_data[table_data['PRODUCT_TYPE_EN'] == 'Cooking Gear']

grouped_data = filtered_data.groupby('PRODUCT_NAME').agg(
    TOTAL_QUANTITY=('QUANTITY', 'sum'),
    AVERAGE_SALES_PRICE=('UNIT_SALE_PRICE', 'mean')
).reset_index()

pandas.DataFrame(grouped_data)



,PRODUCT_NAME,TOTAL_QUANTITY,AVERAGE_SALES_PRICE
0,TrailChef Canteen,26544,11.979418
1,TrailChef Cook Set,10878,51.687926
2,TrailChef Cup,27418,6.976490
3,TrailChef Deluxe Cook Set,3110,121.217638
4,TrailChef Double Flame,12476,132.672882
5,TrailChef Kettle,8826,12.201235
6,TrailChef Kitchen Kit,22206,22.345975
7,TrailChef Single Flame,8504,63.827622
8,TrailChef Utensils,13776,17.294151
9,TrailChef Water Bag,36738,6.032506


Geef een overzicht met daarin per land de naam van het land, de naam van de stad waar de verkoopafdeling is gevestigd (noem de kolomnaam in het overzicht ‘verkoper’) en het aantal steden waar zich klanten bevinden in dat land (noem de kolomnaam in het overzicht ‘klanten’) (3 kolommen, 29 rijen) 

In [31]:
table_data = get_table_data(['retailer_site', 'country'])

grouped_data = table_data.groupby('COUNTRY').agg(
    verkoper=('CITY', 'first'),
    klanten=('CITY', 'nunique')
).reset_index()

pandas.DataFrame(grouped_data)


,COUNTRY,verkoper,klanten
0,Australia,Melbourne,6
1,Austria,Wien,9
2,Belgium,Zaventem,8
3,Brazil,São Paulo,1
4,Canada,Winnipeg,16
5,China,Beijing,7
6,Denmark,København,5
7,Finland,Tampere,5
8,France,Paris,14
9,Germany,Frankfurt,18


## Pythonvertalingen van SUBSELECT en UNION met o.a. for-loops

Geef een overzicht met daarin de voornaam en de achternaam van de medewerkers die nog nooit wat hebben verkocht (2 kolommen, 25 rijen) 

In [32]:
sales_staff_data = get_table_data('sales_staff')
order_header_data = get_table_data('order_header')

merged_data = sales_staff_data.merge(order_header_data, on='SALES_STAFF_CODE', how='left')
filter = merged_data[merged_data['ORDER_NUMBER'].isna()]

filter[['FIRST_NAME', 'LAST_NAME']]

,FIRST_NAME,LAST_NAME
0,Denis,Pagé
320,Else,Mörike
321,Frank,Fuchs
647,Fritz,Hirsch
648,Jörg,Kunze
728,Maria,Iacobucci
946,Kick,Kalkman
1309,Karin,Bergström
1395,Sally,White
1421,Frank,Bretton


Geef een overzicht met daarin het aantal producten waarvan de marge lager is dan de gemiddelde marge van alle producten samen. Geef in het overzicht tevens aan wat de gemiddelde marge is van dit aantal producten waarvan de marge lager dan de gemiddelde marge van alle producten samen is. (1 kolom, 2 rijen) 

In [33]:
product_data = get_table_data('product')

gem_marge = product_data['MARGIN'].mean()
filter = product_data['MARGIN'] < gem_marge

print(filter.sum())

59


Geef een overzicht met daarin de namen van de producten die voor meer dan 500 (verkoopprijs) zijn verkocht maar nooit zijn teruggebracht. (1 kolom, 13 rijen) 

In [34]:
product_data = get_table_data('product')
order_details_data = get_table_data('order_details')
return_item_data = get_table_data('returned_item')

merged_data = product_data.merge(order_details_data, on='PRODUCT_NUMBER', how='left')
merged_data2 = merged_data.merge(return_item_data, on='ORDER_DETAIL_CODE', how='left')
filter = merged_data2.loc[(merged_data2['RETURN_REASON_CODE'].notna()) & (merged_data2['UNIT_PRICE'] > 500)]
filter = filter[['PRODUCT_NAME']].drop_duplicates()

pandas.DataFrame(filter['PRODUCT_NAME'])

,PRODUCT_NAME
3961,Star Dome
4486,Star Gazer 2
4723,Star Gazer 3
5093,Star Gazer 6
15219,Husky Rope 200
33224,Hailstorm Steel Irons
33663,Hailstorm Titanium Irons
34040,Lady Hailstorm Steel Irons
34351,Lady Hailstorm Titanium Irons
34492,Hailstorm Titanium Woods Set


Geef een overzicht met daarin per (achternaam van) medewerker of hij/zij manager is of niet, door deze informatie toe te voegen als extra 'Ja/Nee'-kolom.<br>
Hint: gebruik een for-loop waarin je o.a. bepaalt of het woord 'Manager' in de functie (position_en) staat. (2 kolommen, 102 rijen).

In [35]:
sales_staff_data = get_table_data('sales_staff')

is_manager = []

for position in sales_staff_data['POSITION_EN']:
    if 'Manager' in str(position):
        is_manager.append('Ja')
    else:
        is_manager.append('Nee')

sales_staff_data['IS_MANAGER'] = is_manager
overview = sales_staff_data[['LAST_NAME', 'IS_MANAGER']]

pandas.DataFrame(overview)

,LAST_NAME,IS_MANAGER
0,Pagé,Ja
1,Michel,Nee
2,Clermont,Nee
3,Jauvin,Nee
4,Wiesinger,Nee
...,...,...
97,Laermans,Nee
98,De Crée,Nee
99,Lattrez,Nee
100,Seefelder,Nee


Met de onderstaande code laat je Python het huidige jaar uitrekenen.

In [36]:
from datetime import date
date.today().year

2025

Met de onderstaande code selecteer je op een bepaald jaartal uit een datum.

In [37]:
from datetime import datetime

date_str = '16-8-2013'
date_format = '%d-%m-%Y'
date_obj = datetime.strptime(date_str, date_format)

date_obj.year

2013

Geef met behulp van bovenstaande hulpcode een overzicht met daarin op basis van het aantal jaar dat iemand in dienst is of een medewerker ‘kort in dienst’ (minder dan 25 jaar in dienst) of een ‘lang in dienst’ (groter gelijk dan 12 jaar in dienst) is. Geef daarbij per medewerker in een aparte kolom zowel ‘kort in dienst’ als ‘lang in dienst’ aan. Gebruik (wederom) een for-loop.<br>
(2 kolommen, 102 rijen) 

In [38]:
from datetime import date, datetime 

sales_staff_data = get_table_data('sales_staff')

in_dienst = []

current_year = date.today().year
date_format = '%Y-%m-%d'

for hire_date in sales_staff_data['DATE_HIRED']: 
    hire_year = datetime.strptime(hire_date, date_format).year
    if (current_year - hire_year) < 25:
        in_dienst.append('kort in dienst')
    else:
        in_dienst.append('lang in dienst')

sales_staff_data['DIENST_TIJD'] = in_dienst
overview = sales_staff_data[['LAST_NAME', 'DIENST_TIJD']]

pandas.DataFrame(overview)

,LAST_NAME,DIENST_TIJD
0,Pagé,lang in dienst
1,Michel,lang in dienst
2,Clermont,lang in dienst
3,Jauvin,lang in dienst
4,Wiesinger,lang in dienst
...,...,...
97,Laermans,lang in dienst
98,De Crée,lang in dienst
99,Lattrez,lang in dienst
100,Seefelder,lang in dienst


## Van Jupyter Notebook naar Pythonproject

1. Richt de map waarin jullie tot nu toe hebben gewerkt in volgens de mappenstructuur uit de slides.
2. Maak van de ontstane mappenstructuur een Pythonproject dat uitvoerbaar is vanuit de terminal. Maak daarin een .py-bestand dat minstens 5 antwoorden uit dit notebook (in de vorm van een DataFrame) exporteert naar Excelbestanden. Alle notebooks mogen als notebook blijven bestaan.
3. Zorg ervoor dat dit Pythonproject zijn eigen repo heeft op Github. Let op: je virtual environment moet <b><u>niet</u></b> meegaan naar Github.

Je mag tijdens dit proces je uit stap 1 ontstane mappenstructuur aanpassen, zolang je bij het beoordelingsmoment kan verantwoorden wat de motivatie hierachter is. De slides verplichten je dus nergens toe.